In [1]:
import igraph as ig
from igraph import *
import numpy as np
import math
###################################################
def init_IMDB_graph():
    N=21 #Number_of_vertices
    g=Graph()
    g.add_vertices(N)
    g.add_edges([(0,1), (0,2), (0,4), (0,5), (0,6), (0,11),(0,13), (0,17), (0,20), (1,2), (2,4), (2,5), (2,6), (2,11), (2,13), (2,17), (2,20),
                (3,4), (3,5), (3,9), (4,5), (4,6), (4,11), (4,13), (4,17), (4,20), (5,6), (5,11), (5,13), (5,17), (5,20), (6,7), (6,8), (6,9),
                (6,10), (6,16), (6,17), (6,20), (8,9), (8,10), (11,12), (11,13), (11,17), (11,20), (13,14), (13,15), (13,17),(13,20), (17,18),
                (17,20), (19,20)])
    g.vs["table_names"] = ["title","kind_type","aka_title","info_type","movie_info","movie_info_idx","cast_info","role_type","name",
                           "person_info","aka_name","movie_keyword","keyword","movie_companies","company_type","company_name","char_name",
                           "complete_cast","comp_cast_type","link_type","movie_link"]
    g.vs["size"]=[4189165, 7, 515643, 113, 257641243, 2280011, 61375706, 12, 6278675, 4048009, 1279133, 7200278, 222746, 4784207, 4,
                   352569, 2763179, 135086, 4, 18, 2530092]
    g.vs["lable"]=["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
    g.es["selectivity"]=[0.142, 2.01e-7, 2.14e-7, 2.16e-7, 2.22e-7, 2.11e-7, 2.17e-7, 2.20e-7, 6.88e-9, 0.1428, 1.37e-6, 9.26e-7, 2.01e-7,
                        2.73e-6, 1.25e-6, 2.13e-6, 2.74e-6, 8.84e-3, 8.84e-3, 8.84e-3, 6.00e-8, 9.01e-8, 2.38e-7, 3.08e-8, 1.58e-7, 5.08e-6,
                        4.01e-7, 9.84e-7, 6.25e-7, 9.75e-7, 8.91e-7, 3.26e-3, 1.59e-7, 1.38e-6, 1.60e-7, 1.62e-8, 1.58e-8, 7.58e-7, 1.59e-7,
                        1.59e-7, 4.48e-6, 1.75e-6, 2.57e-6, 3.28e-6, 0.25, 2.83e-6, 1.12e-5, 1.28e-6, 2.57e-6, 2.57e-6, 5.55e-2]
    sel=np.zeros((N,N))
    pred=np.zeros((N,N), np.bool)
    
    for edge in g.es:
        s=edge.source
        t=edge.target
        sel[t][s]=sel[s][t]=edge["selectivity"]
        pred[t][s]=pred[s][t]=True
        
    return N,g,sel,pred

###################################################
def init_test_graph():
    N=7 #Number_of_vertices
    g=Graph()
    g.add_vertices(N)
    g.add_edges([(0,1), (0,6), (1,2), (1,4), (2,3), (2,4), (3,4), (5,6)]) # id = label-1
    g.vs["size"]=[10, 10, 10, 10, 10, 10, 10]
    g.vs["lable"]=["1","2","3","4","5","6","7"]
    g.es["selectivity"]=[0.7, 0.9, 0.3, 0.4, 0.5, 0.6, 0.2, 0.8]
    sel=np.zeros((N,N))
    pred=np.zeros((N,N), np.bool)
    
    for edge in g.es:
        s=edge.source
        t=edge.target
        sel[t][s]=sel[s][t]=edge["selectivity"]
        pred[t][s]=pred[s][t]=True

    return N,g,sel,pred
                    
###################################################
def GOO_Algorithm (N,g,sel,pred):
    
    for n in range (N-1,0,-1):
        print ("................................................S T E P " , N-n , "................................................")
        min_value= -1
        for edge in g.es:          #find min value
            s=edge.source
            t=edge.target
            if s == t:
                continue
            val= g.vs[s]["size"] * g.vs[t]["size"] * edge["selectivity"]
            if min_value == -1:
                min_value = val
                i=s
                j=t
            elif val < min_value:
                min_value = val
                i=s
                j=t
                
                
        print ("MIN: ", min_value)
        print ("S:", g.vs[i]["lable"])
        print( "T:" , g.vs[j]["lable"])                
        print("---------------------------------------------")
        print ("BEFORE CHANGES....")
        print (sel)
        print(pred)
        v_id=0
        for vertice in g.vs:
            print("ID:" ,v_id , "LABLE:" , vertice["lable"] , "SIZE:" , vertice["size"] )
            v_id +=1
        for edge in g.es:
            s=edge.source
            t=edge.target
            print("S:" , g.vs[s]["lable"] , "," , "T:" , g.vs[t]["lable"] , "," , "SELECTIVITY:" , edge["selectivity"] )
        
        
        sel[i][j] = sel[j][i] = 0
        pred[i][j] = pred[j][i] = False
        g.delete_edges(g.get_eid(i,j))
        
        for k in g.neighbors(i):
            if pred[k][j] :
                
                sel[i][k] = sel[k][i] = sel[i][k] * sel[j][k]
                g.es[g.get_eid(i,k)]["selectivity"]= sel[i][k]
                
                sel[k][j] = sel[j][k] = 0
                pred[k][j] = pred[j][k] = False
                g.delete_edges(g.get_eid(k,j))
                
        for m in g.neighbors(j):
            
            pred[i][m] = pred[m][i] = True
            sel[i][m] = sel[m][i] = sel[j][m]
            g.add_edge(i,m)
            g.es[g.get_eid(i,m)]["selectivity"]= sel[j][m]
            
            sel[m][j] = sel[j][m] = 0
            pred[m][j] = pred[j][m] = False
            g.delete_edges(g.get_eid(m,j))
            
                
        g.vs[i]["lable"] =  g.vs[i]["lable"] + " ⨝ " + g.vs[j]["lable"]           # set lable
        g.vs[i]["size"]= math.ceil(min_value)                                                # set size
        
        sel = np.delete(sel, j , axis=1)
        sel = np.delete(sel, j, axis=0)
        pred = np.delete(pred, j , axis=1)
        pred = np.delete(pred, j, axis=0)
        g.delete_vertices(j)
        
                
        print("---------------------------------------------")
        print ("AFTER CHANGES....")
        print (sel)
        print(pred)
        v_id=0
        for vertice in g.vs:
            print("ID:" ,v_id , "," , "LABLE:" , vertice["lable"] , "," , "SIZE:" , vertice["size"] )
            v_id +=1
        for edge in g.es:
            s=edge.source
            t=edge.target
            print("S:" , g.vs[s]["lable"] , "," , "T:" , g.vs[t]["lable"] , "," , "SELECTIVITY:" , edge["selectivity"] )
        
###################################################
N,g,sel,pred=init_test_graph()
GOO_Algorithm(N,g,sel,pred)

................................................S T E P  1 ................................................
MIN:  20.0
S: 4
T: 5
---------------------------------------------
BEFORE CHANGES....
[[0.  0.7 0.  0.  0.  0.  0.9]
 [0.7 0.  0.3 0.  0.4 0.  0. ]
 [0.  0.3 0.  0.5 0.6 0.  0. ]
 [0.  0.  0.5 0.  0.2 0.  0. ]
 [0.  0.4 0.6 0.2 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.8]
 [0.9 0.  0.  0.  0.  0.8 0. ]]
[[False  True False False False False  True]
 [ True False  True False  True False False]
 [False  True False  True  True False False]
 [False False  True False  True False False]
 [False  True  True  True False False False]
 [False False False False False False  True]
 [ True False False False False  True False]]
ID: 0 LABLE: 1 SIZE: 10
ID: 1 LABLE: 2 SIZE: 10
ID: 2 LABLE: 3 SIZE: 10
ID: 3 LABLE: 4 SIZE: 10
ID: 4 LABLE: 5 SIZE: 10
ID: 5 LABLE: 6 SIZE: 10
ID: 6 LABLE: 7 SIZE: 10
S: 1 , T: 2 , SELECTIVITY: 0.7
S: 1 , T: 7 , SELECTIVITY: 0.9
S: 2 , T: 3 , SELECTIVITY: 0.3
S: 2 , T: 5